In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('ggplot')

In [2]:
train_df = pd.read_csv('dataset/telstra/train.csv')
print 'train_df', train_df.shape
test_df = pd.read_csv('dataset/telstra/test.csv')
print 'test_df', test_df.shape
event_df = pd.read_csv('dataset/telstra/event_type.csv')
print 'event_df', event_df.shape
log_df = pd.read_csv('dataset/telstra/log_feature.csv')
print 'log_df', log_df.shape
resource_df = pd.read_csv('dataset/telstra/resource_type.csv')
print 'resource_df', resource_df.shape
severity_df = pd.read_csv('dataset/telstra/severity_type.csv')
print 'severity_df', severity_df.shape

train_df (7381, 3)
test_df (11171, 2)
event_df (31170, 2)
log_df (58671, 3)
resource_df (21076, 2)
severity_df (18552, 2)


In [3]:
# Set 'id' as index
train_df.set_index('id', drop=True, inplace=True)
test_df.set_index('id', drop=True, inplace=True)
event_df.set_index('id', drop=True, inplace=True)
log_df.set_index('id', drop=True, inplace=True)
resource_df.set_index('id', drop=True, inplace=True)
severity_df.set_index('id', drop=True, inplace=True)

In [4]:
train = pd.merge(train_df, event_df, left_index=True, right_index=True, how='inner')
train = pd.merge(train, resource_df, left_index=True, right_index=True, how='inner')
train = pd.merge(train, severity_df, left_index=True, right_index=True, how='inner')
train

,location,fault_severity,event_type,resource_type,severity_type
id,,,,,
1,location 601,1,event_type 11,resource_type 8,severity_type 1
1,location 601,1,event_type 11,resource_type 6,severity_type 1
1,location 601,1,event_type 13,resource_type 8,severity_type 1
1,location 601,1,event_type 13,resource_type 6,severity_type 1
5,location 460,0,event_type 34,resource_type 2,severity_type 2
5,location 460,0,event_type 35,resource_type 2,severity_type 2
6,location 332,1,event_type 34,resource_type 2,severity_type 2
8,location 243,0,event_type 34,resource_type 2,severity_type 2
8,location 243,0,event_type 35,resource_type 2,severity_type 2
